<a href="https://colab.research.google.com/github/jiangxueqi/LLM-quickstart/blob/main/homework/homework7_1_%E4%BD%BF%E7%94%A8GPTQ%E9%87%8F%E5%8C%96OPT_6_7B%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/AI大模型微调训练营/6_实战Transformers模型微调/requirements.txt"

#1、使用默认数据集C4

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_id = "facebook/opt-6.7b"

quantization_config = GPTQConfig(
     bits=4,     # 量化精度
     group_size=128, # 超参数，默认128
     dataset="c4",  # 使用的数据集
     desc_act=False,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
quant_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map='auto')

# 2、检查量化模型正确性

In [ ]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

#3、使用GPU加载量化后的模型并生成文本

In [ ]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)    # return_tensors="pt"，返回的tensor是pytoch的数据样式

out = quant_model.generate(**inputs, max_new_tokens=64)  # max_new_tokens="64"，生成的内容最大长度
print(tokenizer.decode(out[0], skip_special_tokens=True))  # skip_special_tokens，有多句话时候处理分割符【CLS】